# Árbol de Segmentos para Varianzas de Intervalos

Este notebook implementa un árbol de segmentos que mantiene y consulta la varianza de intervalos de manera eficiente usando la fórmula compacta.

**Fórmula**: `Var(X) = E[X²] - (E[X])²`

## Clase Node: El Nodo del Árbol

Cada nodo almacena:
- `suma`: Σxi
- `suma_cuadrados`: Σxi²
- `count`: cantidad de elementos

In [ ]:
class Node:
    """
    Clase que representa un nodo del árbol de segmentos.

    Atributos:
        suma: Suma de todos los elementos en el intervalo
        suma_cuadrados: Suma de los cuadrados de los elementos
        count: Cantidad de elementos en el intervalo
    """

    def __init__(self, suma=0, suma_cuadrados=0, count=0):
        self.suma = suma
        self.suma_cuadrados = suma_cuadrados
        self.count = count

    def get_variance(self):
        """Calcula la varianza usando la fórmula compacta: Var(X) = E[X²] - (E[X])²"""
        if self.count == 0:
            return 0

        media = self.suma / self.count
        media_cuadrados = self.suma_cuadrados / self.count
        varianza = media_cuadrados - (media * media)
        return max(0, varianza)

    def get_mean(self):
        """Calcula la media del intervalo."""
        if self.count == 0:
            return 0
        return self.suma / self.count

    def __repr__(self):
        return f"Node(suma={self.suma}, suma_cuad={self.suma_cuadrados}, count={self.count}, var={self.get_variance():.4f})"

## Clase SegmentTreeVariance: El Árbol Completo

Operaciones principales:
- Construcción: O(n)
- Consulta: O(log n)
- Actualización: O(log n)

In [ ]:
class SegmentTreeVariance:
    """
    Árbol de Segmentos que mantiene varianzas de intervalos.
    """

    def __init__(self, arr):
        """Inicializa el árbol de segmentos con un array dado."""
        self.arr = arr[:]
        self.n = len(arr)
        self.tree = [Node() for _ in range(4 * self.n)]

        if self.n > 0:
            self._build(0, 0, self.n - 1)

    def _merge(self, left_node, right_node):
        """Combina dos nodos hijos para crear el nodo padre."""
        return Node(
            suma=left_node.suma + right_node.suma,
            suma_cuadrados=left_node.suma_cuadrados + right_node.suma_cuadrados,
            count=left_node.count + right_node.count
        )

    def _build(self, node_idx, left, right):
        """Construye el árbol de segmentos de manera recursiva."""
        if left == right:
            value = self.arr[left]
            self.tree[node_idx] = Node(
                suma=value,
                suma_cuadrados=value * value,
                count=1
            )
            return

        mid = (left + right) // 2
        left_child = 2 * node_idx + 1
        right_child = 2 * node_idx + 2

        self._build(left_child, left, mid)
        self._build(right_child, mid + 1, right)

        self.tree[node_idx] = self._merge(
            self.tree[left_child],
            self.tree[right_child]
        )

    def update(self, index, new_value):
        """Actualiza el valor en una posición específica. Complejidad: O(log n)"""
        if index < 0 or index >= self.n:
            raise IndexError(f"Índice {index} fuera de rango [0, {self.n-1}]")

        self.arr[index] = new_value
        self._update_recursive(0, 0, self.n - 1, index, new_value)

    def _update_recursive(self, node_idx, left, right, target_idx, new_value):
        """Función auxiliar recursiva para actualizar el árbol."""
        if left == right:
            self.tree[node_idx] = Node(
                suma=new_value,
                suma_cuadrados=new_value * new_value,
                count=1
            )
            return

        mid = (left + right) // 2
        left_child = 2 * node_idx + 1
        right_child = 2 * node_idx + 2

        if target_idx <= mid:
            self._update_recursive(left_child, left, mid, target_idx, new_value)
        else:
            self._update_recursive(right_child, mid + 1, right, target_idx, new_value)

        self.tree[node_idx] = self._merge(
            self.tree[left_child],
            self.tree[right_child]
        )

    def query_variance(self, query_left, query_right):
        """Consulta la varianza de un rango. Complejidad: O(log n)"""
        if query_left < 0 or query_right >= self.n or query_left > query_right:
            raise ValueError(f"Rango inválido [{query_left}, {query_right}]")

        result_node = self._query_recursive(0, 0, self.n - 1, query_left, query_right)
        return result_node.get_variance()

    def query_mean(self, query_left, query_right):
        """Consulta la media de un rango. Complejidad: O(log n)"""
        if query_left < 0 or query_right >= self.n or query_left > query_right:
            raise ValueError(f"Rango inválido [{query_left}, {query_right}]")

        result_node = self._query_recursive(0, 0, self.n - 1, query_left, query_right)
        return result_node.get_mean()

    def query_sum(self, query_left, query_right):
        """Consulta la suma de un rango. Complejidad: O(log n)"""
        if query_left < 0 or query_right >= self.n or query_left > query_right:
            raise ValueError(f"Rango inválido [{query_left}, {query_right}]")

        result_node = self._query_recursive(0, 0, self.n - 1, query_left, query_right)
        return result_node.suma

    def _query_recursive(self, node_idx, left, right, query_left, query_right):
        """Función auxiliar recursiva para consultar un rango."""
        # Sin solapamiento
        if query_right < left or query_left > right:
            return Node()

        # Solapamiento total
        if query_left <= left and right <= query_right:
            return self.tree[node_idx]

        # Solapamiento parcial
        mid = (left + right) // 2
        left_child = 2 * node_idx + 1
        right_child = 2 * node_idx + 2

        left_result = self._query_recursive(left_child, left, mid, query_left, query_right)
        right_result = self._query_recursive(right_child, mid + 1, right, query_left, query_right)

        return self._merge(left_result, right_result)

    def get_array(self):
        """Retorna una copia del array actual."""
        return self.arr[:]

    def print_tree(self, node_idx=0, level=0, left=None, right=None):
        """Imprime el árbol de segmentos de manera visual."""
        if left is None:
            left = 0
        if right is None:
            right = self.n - 1

        if left > right:
            return

        indent = "  " * level
        node = self.tree[node_idx]
        print(f"{indent}[{left},{right}]: {node}")

        if left < right:
            mid = (left + right) // 2
            self.print_tree(2 * node_idx + 1, level + 1, left, mid)
            self.print_tree(2 * node_idx + 2, level + 1, mid + 1, right)

## Ejemplo 1: Construcción y Consultas Básicas

In [ ]:
print("="*70)
print("EJEMPLO 1: Construcción y consultas básicas")
print("="*70)

arr = [4, 8, 6, 2, 10, 12, 14, 16]
print(f"Array original: {arr}")

st = SegmentTreeVariance(arr)
print("\nConsultas de varianza:")

ranges = [(0, 3), (2, 5), (0, 7), (4, 7)]
for l, r in ranges:
    variance = st.query_variance(l, r)
    mean = st.query_mean(l, r)
    suma = st.query_sum(l, r)
    elements = arr[l:r+1]
    print(f"  Rango [{l}, {r}]: elementos={elements}")
    print(f"    -> Suma={suma:.2f}, Media={mean:.2f}, Varianza={variance:.4f}")

## Ejemplo 2: Actualización de Valores

In [ ]:
print("\n" + "="*70)
print("EJEMPLO 2: Actualización de valores")
print("="*70)

print(f"Array antes: {st.get_array()}")
print(f"Varianza del rango [0, 3] antes: {st.query_variance(0, 3):.4f}")

st.update(1, 4)  # Cambiar arr[1] de 8 a 4
print(f"\nArray después de update(1, 4): {st.get_array()}")
print(f"Varianza del rango [0, 3] después: {st.query_variance(0, 3):.4f}")

## Ejemplo 3: Verificación con Cálculo Manual

In [ ]:
print("\n" + "="*70)
print("EJEMPLO 3: Verificación de la fórmula compacta")
print("="*70)

test_arr = [2, 4, 4, 4, 5, 5, 7, 9]
st2 = SegmentTreeVariance(test_arr)
print(f"Array de prueba: {test_arr}")

# Calcular varianza manualmente
n = len(test_arr)
suma_manual = sum(test_arr)
suma_cuad_manual = sum(x*x for x in test_arr)
media_manual = suma_manual / n
media_cuad_manual = suma_cuad_manual / n
var_manual = media_cuad_manual - (media_manual ** 2)

var_arbol = st2.query_variance(0, n-1)

print(f"\nCálculo manual:")
print(f"  Suma = {suma_manual}")
print(f"  Suma de cuadrados = {suma_cuad_manual}")
print(f"  Media = {media_manual}")
print(f"  E[X²] = {media_cuad_manual}")
print(f"  Varianza = E[X²] - (E[X])² = {media_cuad_manual} - {media_manual**2} = {var_manual:.6f}")
print(f"\nVarianza del árbol: {var_arbol:.6f}")
print(f"Diferencia: {abs(var_manual - var_arbol):.10f}")

## Ejemplo 4: Múltiples Actualizaciones

In [ ]:
print("\n" + "="*70)
print("EJEMPLO 4: Múltiples actualizaciones y consultas")
print("="*70)

arr3 = [10, 20, 30, 40, 50]
st3 = SegmentTreeVariance(arr3)
print(f"Array inicial: {st3.get_array()}")
print(f"Varianza [0, 4]: {st3.query_variance(0, 4):.4f}")

updates = [(0, 30), (4, 30)]  # Hacer el array más uniforme
for idx, val in updates:
    st3.update(idx, val)
    print(f"\nDespués de update({idx}, {val}): {st3.get_array()}, Varianza [0, 4]: {st3.query_variance(0, 4):.4f}")